In [2]:
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

from pydub import AudioSegment
from pydub.playback import play

/opt/venv/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# Function to mix and play multiple WAV files
def mix_and_play_wav_files(file_paths, volumes=None, loops=None, offsets=None, panning=None):
    mixed = None

    for i, file_path in enumerate(file_paths):
        sound = AudioSegment.from_wav(file_path)
        duration = len(sound) / 1000.0
        # waveform_path = plot_waveform(file_path)
        
        # Apply volume adjustment if provided
        if volumes and i < len(volumes):
            sound += volumes[i]  # Adjust volume in dB
        
        # Apply looping if provided
        if loops and i < len(loops):
            sound *= loops[i]  # Repeat sound
        
        # Apply panning if provided
        if panning and i < len(panning):
            sound = sound.pan(panning[i])  # Pan sound
        
        # Determine the position for this sound
        position = 0
        if offsets and i < len(offsets):
            position = offsets[i]
        
        # Overlay the sound at the specified position
        if mixed is None:
            # If this is the first sound, just use it as the start of the mix
            mixed = sound
        else:
            # Extend the mixed track if necessary to ensure there's enough duration for the offset
            if position + len(sound) > len(mixed):
                mixed += AudioSegment.silent(duration=(position + len(sound) - len(mixed)))
            
            mixed = mixed.overlay(sound, position=position)

    # Play the mixed audio
    if mixed:
        play(mixed)

In [4]:
model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=8)

/opt/venv/lib/python3.11/site-packages/audiocraft/models/musicgen.py:127: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-melody
  warnings.warn(
/opt/venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [5]:
def generate(descriptions=['sad boy jazz on guitar with bass'], melody=None):
    """
    
    Args:
        descriptions (list of str): List of prompt inputs into model
        melody (.wav or .mp3 file): Melody to input into model

    """
    sr = 44100
    

    # melody, sr = torchaudio.load(melody)
    # generates using the melody from the given audio and the provided descriptions.
    if melody:
        # sr = melody.
        wav = model.generate_with_chroma(descriptions, melody[None].expand(3, -1, -1), sr)
    else:
        wav = model.generate(descriptions, sr)

    for idx, one_wav in enumerate(wav):
        # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
        audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness")

generate()

Unable to find python bindings at /usr/local/dcgm/bindings/python3. No data will be captured.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this appli

: 

: 

: 

In [ ]:
# List of WAV file paths to mix
wav_files = [
    '0.wav'
]
volumes = [1]  # Adjust volume of each file in dB
loops = [1]     # Number of times to loop each file
offsets = [0]  # Offset for each file in milliseconds
mix_and_play_wav_files(wav_files, volumes, loops, offsets)

Input #0, wav, from '/var/folders/rn/83xs8vk54_bg2mz34vvwp9w40000gn/T/tmph3692pzs.wav':
  Duration: 00:00:08.00, bitrate: 512 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
